# Data 101 — Module 5, Session 3
## Wrangling & Summarizing Data (Demo Notebook)

This notebook mirrors the Session 3 slides:
- Import raw data
- Inspect and detect issues
- Handle missing values
- Fix data types
- Remove duplicates and inconsistencies
- Detect potential outliers
- Create derived columns
- Aggregate, group, and pivot
- Export cleaned data

## Import and Read Raw Data

In [ ]:
import pandas as pd, numpy as np
df = pd.read_csv("./data/students_messy.csv")
print("Shape:", df.shape)

In [ ]:
display(df.head())

In [ ]:
print("\nInfo:")
display(df.info())

## Inspect Missingness and Basic Profiling

In [ ]:
missing_counts = df.isnull().sum()
print("Missing values per column:\n", missing_counts)

In [ ]:
df.isnull().mean().plot.bar()

In [ ]:
print("\nDescribe numeric columns:")
display(df.describe())

In [ ]:
print("\nUnique values in 'Major' (notice inconsistencies):")
print(sorted(df["Major"].astype(str).unique()))

## Handle Missing Values

In [ ]:
df["GPA"] = pd.to_numeric(df["GPA"].astype(str).str.strip(), errors="coerce")
df["GPA"] = df["GPA"].fillna(df["GPA"].mean())
print("Post-imputation missingness:")
print(df.isnull().sum())
display(df.head())

In [ ]:
df["Hours_Studied"] = pd.to_numeric(df["Hours_Studied"], errors="coerce")
df["Hours_Studied"] = df["Hours_Studied"].fillna(df["Hours_Studied"].median())
print("Post-imputation missingness:")
print(df.isnull().sum())
display(df.head())

In [ ]:
df["ExamDate"] = pd.to_datetime(df["ExamDate"], format="%Y-%m-%d", errors="coerce")
print(df.isnull().sum())
display(df.head())

In [ ]:
df = df.dropna(subset=["ExamDate"])
print(df.isnull().sum())
display(df.head())

In [ ]:
dft = pd.DataFrame({"Score": [10, None, None, 20, None, 30]})
print(dft)

In [ ]:
print(dft.ffill())

In [ ]:
print(dft.bfill())

## Fix Types and Standardize Categories

In [ ]:
df["Major"] = (df["Major"].astype(str).str.strip().str.lower()
    .replace({"computer science":"cs","mathematics":"math","phys":"physics"})
    .str.replace(r"\s+", " ", regex=True))
df["Major"] = df["Major"].map(lambda x: x.title())
df["Major"] = df["Major"].astype("category")
df["Gender"] = df["Gender"].astype("category")
display(df.sample(5, random_state=0))
print(df.dtypes)

## Remove Duplicates

In [ ]:
print("Duplicate rows before:", df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True)
print("Duplicate rows after:", df.duplicated().sum())
print("New shape:", df.shape)

## Potential Outliers (IQR Method)

In [ ]:
def iqr_filter(series, k=1.5):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    return q1 - k*iqr, q3 + k*iqr

for col in ["GPA", "Hours_Studied"]:
    low, high = iqr_filter(df[col])
    mask = ~df[col].between(low, high)
    print(f"{col}: {mask.sum()} potential outliers outside [{low:.2f}, {high:.2f}]")
    display(df.loc[mask, ["Name","Major","Gender",col]].head())

## Create Derived Columns

In [ ]:
df["Passed"] = np.where(df["GPA"] >= 2.0, "Yes", "No")
df["Study_Efficiency"] = df["GPA"] / df["Hours_Studied"].replace(0, np.nan)
df["ExamMonth"] = df["ExamDate"].dt.month
df["ExamMonth"] = df["ExamMonth"].fillna(-1).astype(int)
display(df.head())

## Aggregation and Grouping

In [ ]:
summary = df.agg({
    "GPA": ["mean","median","min","max","std"],
    "Hours_Studied": ["mean","median","min","max","std"]
})
display(summary)

In [ ]:
grouped = (df.groupby(["Major","Gender"], observed=True)
    .agg(GPA_mean=("GPA","mean"), GPA_std=("GPA","std"),
         Hours_mean=("Hours_Studied","mean"))
    .sort_values(["GPA_mean"], ascending=False))
display(grouped.head(10))

## Pivot Table

In [ ]:
pivot = pd.pivot_table(df, values="GPA", index="Major", columns="Gender", aggfunc="mean", observed=True)
display(pivot)

## Optional: GPA Distribution Plot

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
df["GPA"].plot(kind="hist", bins=10)
plt.title("GPA Distribution")
plt.xlabel("GPA")
plt.ylabel("Count")
plt.grid(True)
plt.show()

## Export Cleaned Data

In [ ]:
out_path = "./data/students_clean.csv"
df.to_csv(out_path, index=False)
print("Saved:", out_path)

## Summary
- Imported raw data and inspected structure and missingness.
- Cleaned missing values, fixed data types, standardized categories.
- Removed duplicates, flagged potential outliers via IQR.
- Engineered features and summarized with aggregation, groupby, and pivot tables.
- Exported a reproducible cleaned dataset.